In [1]:
!pip install -q langchain-openai langchain-core requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 1.4 MB/s eta 0:00:00


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [3]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [4]:
openai_base_url = "https://openrouter.ai/api/v1"

In [5]:
llm = ChatOpenAI(
    openai_api_base=openai_base_url,
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model_name="deepseek/deepseek-chat-v3-0324:free",
    temperature=0.7
)

In [6]:
# create tool

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 number a and b this tool returns their product"""
  return a*b

In [7]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [8]:
multiply.name

'multiply'

In [9]:
multiply.description

'Given 2 number a and b this tool returns their product'

In [10]:
multiply.args_schema

langchain_core.utils.pydantic.multiply

In [11]:
# tool binding
# we already have the llm defined above

In [12]:
llm_with_tools = llm.bind_tools([multiply])
# now this llm_with_tool is a llm but also have tools

In [13]:
# tool calling

In [14]:
llm_with_tools.invoke("How are you?")
# in this we didn't see out tool bcoz it didn't feel to call the tool

AIMessage(content="I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with anything you need! How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 166, 'total_tokens': 202, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-chat-v3-0324:free', 'system_fingerprint': None, 'id': 'gen-1750133741-VQYK5h2Jwrrw1zqiljQ4', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--a82d7913-8641-45be-840b-a742da0adb0a-0', usage_metadata={'input_tokens': 166, 'output_tokens': 36, 'total_tokens': 202, 'input_token_details': {}, 'output_token_details': {}})

In [15]:
llm_with_tools.invoke("can you multiply 3 by 10?")
#  in this content is empty but down we have seen tool_calls

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4WtLzqzgRKusz060jq9Nwg', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 171, 'total_tokens': 194, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-chat-v3-0324:free', 'system_fingerprint': None, 'id': 'gen-1750133743-0TVMu5EYdFuOOFweu564', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6d2083f5-6b24-457f-8e3e-57b33dddead9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'call_4WtLzqzgRKusz060jq9Nwg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 171, 'output_tokens': 23, 'total_tokens': 194, 'input_token_details': {}, 'output_token_details': {}})

In [16]:
llm_with_tools.invoke("can you multiply 3 by 10?").tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 10},
  'id': 'call_W7UkwHLCQTOYjd4NPK_w7w',
  'type': 'tool_call'}]

In [17]:
llm_with_tools.invoke("can you multiply 3 by 10?").tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': 'call_OPL_BDU1R-SLFjiWHtx_bg',
 'type': 'tool_call'}

In [18]:
result = llm_with_tools.invoke("can you multiply 3 by 10?")

In [19]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': 'call_TSj7hLK3S9y2xAndCieE-w',
 'type': 'tool_call'}

In [20]:
result.tool_calls[0]['args']

{'a': 3, 'b': 10}

In [21]:
# Tool Execution

In [22]:
multiply.invoke(result.tool_calls[0]['args'])

30

In [23]:
multiply.invoke({'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': 'call_q44E1dvPR9CkfV92-SM4Kw',
 'type': 'tool_call'})
# here we are getting tool message
# and we can send this message to llm based on this llm will produce the result

ToolMessage(content='30', name='multiply', tool_call_id='call_q44E1dvPR9CkfV92-SM4Kw')

In [24]:
multiply.invoke(result.tool_calls[0])

ToolMessage(content='30', name='multiply', tool_call_id='call_TSj7hLK3S9y2xAndCieE-w')

### **Now from staring let's make again and organised**

In [25]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [26]:
# tool binding
llm_with_tools = llm.bind_tools([multiply])


In [27]:
query = HumanMessage('can you multiply 3 with 1000')

In [28]:
messages = [query]

In [29]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [30]:
result = llm_with_tools.invoke(messages)

In [31]:
messages.append(result)

In [32]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pP0rgjTeQ-mQMN8HoLMvyA', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 171, 'total_tokens': 195, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-chat-v3-0324:free', 'system_fingerprint': None, 'id': 'gen-1750133749-gJksSux5aTni4TcBSMzg', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--08c4abd8-3b10-4542-98f2-5967f6ad8dbd-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_pP0rgjTeQ-mQMN8HoLMvyA', 'type': 'tool_call'}], usage_metadata={'input_tokens': 171, 'output_tokens': 24, 'total_tokens': 195, 'input_token_details': {}, 'output_token_details': {}})]

In [33]:
tool_result = multiply.invoke(result.tool_calls[0])

In [34]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='call_pP0rgjTeQ-mQMN8HoLMvyA')

In [35]:
messages.append(tool_result)

In [36]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pP0rgjTeQ-mQMN8HoLMvyA', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 171, 'total_tokens': 195, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-chat-v3-0324:free', 'system_fingerprint': None, 'id': 'gen-1750133749-gJksSux5aTni4TcBSMzg', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--08c4abd8-3b10-4542-98f2-5967f6ad8dbd-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_pP0rgjTeQ-mQMN8HoLMvyA', 'type': 'tool_call'}], usage_metadata={'input_tokens': 171, 'output_tokens': 24, 'total_tokens': 195, 'input_token_details': {}, 'output_token_details': {}}),
 ToolMessag

In [37]:
llm_with_tools.invoke(messages).content

'The product of 3 multiplied by 1000 is 3000.'

### **Let's do the currency convertor**

In [38]:
# we will use the api of ExchangeRate-API
# we will create two tools one for the fecting the api and another one for the multiplication
# for tool must wirte the description

In [51]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency : str, target_currency : str) -> float:
   """
   This function fetches the currency conversion factor between a given base currency and a target currency
   """
   url = f'https://v6.exchangerate-api.com/v6/f88bdcaae3e4adce7f335b59/pair/{base_currency}/{target_currency}'   #this link we got from that website only

   response = requests.get(url)  #sending req to call the api

   return response.json()


@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [52]:
# let's invoke the tool
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1750118401,
 'time_last_update_utc': 'Tue, 17 Jun 2025 00:00:01 +0000',
 'time_next_update_unix': 1750204801,
 'time_next_update_utc': 'Wed, 18 Jun 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.981}

In [53]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [54]:
# tool binding
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [55]:
# now tool calling
messages = [HumanMessage(
    "Step 1: Use `get_conversion_factor` to find the conversion rate between INR and USD.\n"
    "Step 2: Use `convert` to convert 10 INR into USD using the conversion rate from Step 1.\n"
    "Please ensure both tools are used."
)]

In [56]:
messages

[HumanMessage(content='Step 1: Use `get_conversion_factor` to find the conversion rate between INR and USD.\nStep 2: Use `convert` to convert 10 INR into USD using the conversion rate from Step 1.\nPlease ensure both tools are used.', additional_kwargs={}, response_metadata={})]

In [57]:
ai_message = llm_with_tools.invoke(messages)

In [58]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qI1f8aIXRcWFQOllACA1fw', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 308, 'total_tokens': 338, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-chat-v3-0324:free', 'system_fingerprint': None, 'id': 'gen-1750134043-2yGgMBJ8z1Exg1XG3WSa', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--66271fb1-7ed5-4941-a137-bdeffe7bc40d-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'call_qI1f8aIXRcWFQOllACA1fw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 308, 'output_tokens': 30, 'total_tokens': 338, 'input_token_details': {}, 'output_token_details': {}})

In [59]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_qI1f8aIXRcWFQOllACA1fw',
  'type': 'tool_call'}]

In [60]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [61]:
messages

[HumanMessage(content='Step 1: Use `get_conversion_factor` to find the conversion rate between INR and USD.\nStep 2: Use `convert` to convert 10 INR into USD using the conversion rate from Step 1.\nPlease ensure both tools are used.', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1750118401, "time_last_update_utc": "Tue, 17 Jun 2025 00:00:01 +0000", "time_next_update_unix": 1750204801, "time_next_update_utc": "Wed, 18 Jun 2025 00:00:01 +0000", "base_code": "INR", "target_code": "USD", "conversion_rate": 0.01163}', name='get_conversion_factor', tool_call_id='call_qI1f8aIXRcWFQOllACA1fw')]

In [50]:
llm_with_tools.invoke(messages).content

''

In [62]:
# this above is not AI agent bcoz many decisions are taken by us or llm

In [63]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

<ipython-input-63-627313024>:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [64]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?"
}
```

> Finished chain.
